In [1]:
import random
import numpy as np
import pandas as pd
from itertools import combinations
from tqdm.notebook import tqdm
import uuid
import matplotlib.pyplot as plt
import json 
from typing import List, Dict, Tuple, Any
from Levenshtein import distance
from Bio import Align

In [2]:
def create_random_strand(len_strand: int) -> str:
    choices = ['A', 'C', 'T', 'G']
    return "".join([random.choice(choices) for i in range(len_strand)])

def create_spacer_sequence(cycles):
    """ Create motif level label for the model"""

    spacer_sequence = []

    cycle_number = 9
        
    for i in cycles:
        for j in i:
            spacer_sequence.append(cycle_number)
            spacer_sequence.append(j)
            spacer_sequence.append(cycle_number)
        cycle_number += 1

    return spacer_sequence

def reverse_complement(dna: str) -> str:
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement[base] for base in reversed(dna))

### Writing to fasta file

In [60]:

# First let us see if squigulator lets us seperate out into cycle reads, cause that will make life much simpler

n_motifs_per_read = 32
n_unique_motifs = 8
n_spacer_motifs = 8
motifs_per_payload = 4
len_motif = 20
len_link = 10
n_cycles = 8

motif_choices = [create_random_strand(len_motif) for i in range(n_unique_motifs)]
link_choices = [create_random_strand(len_link) for i in range(n_spacer_motifs)]
motif_indices = [1, 2, 3, 4, 5, 6, 7, 8]

In [61]:
n_reads = 100000
reads_base_level = []
reads_motif_level = []
motif_labels = []
motif_picks = list(combinations(motif_indices, 4)) # Storing all possible combinations to draw from

for read in tqdm(range(n_reads)):
    read_base_level = ""
    read_motif_level = []
    for cycle_position in range(n_cycles):
        payload_motifs = random.choice(motif_picks)
        payload_read_base_level = "".join([f"{link_choices[cycle_position]}{motif_choices[i-1]}{link_choices[cycle_position]}" for i in payload_motifs])
        read_base_level += payload_read_base_level
        read_motif_level.append(payload_motifs)
    reads_base_level.append(read_base_level)
    reads_motif_level.append(read_motif_level)
    motif_label = create_spacer_sequence(read_motif_level)
    motif_labels.append(motif_label)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [62]:
ids = [str(uuid.uuid4()) for i in range(n_reads)]

In [63]:
experiment_dict = {
    "motif_choices": motif_choices, 
    "link_choices": link_choices,
    "motif_labels": motif_labels,
    "read_ids": ids,
    "reads_base_level": reads_base_level
}

# write it as a json file to the data path to make sure the run information is validated

with open('info.json', 'w') as f:
    json.dump(experiment_dict, f)

In [64]:
# writing to fasta file

base_filepath = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\squigulator\split.fa"

with open(base_filepath, "w") as f:
    for ind in range(len(ids)):
        f.write(f">>{ids[ind]}\n")
        f.write(reads_base_level[ind] + "\n\n")

In [65]:
base_filepath = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\squigulator\split.fa"
with open(base_filepath, 'r') as f:
    fasta_seq = f.readlines()[1]


### Loading fast5 data after generating via Squigulator

In [10]:

from ont_fast5_api.fast5_interface import get_fast5_file

def get_data_from_fast5(fast5_filepath):
    raw_data_arr = []
    read_ids = []
    with get_fast5_file(fast5_filepath, mode="r") as f5:
        for read in f5.get_reads():
            raw_data = read.get_raw_data()
            raw_data_arr.append(raw_data)
            read_ids.append(read.read_id)
    return raw_data_arr, read_ids


def add_remainder_motif(ptr):

    # Adding the motif if majority of it is present
    # Hardcoded, spacer is 10 and payload is 20 S - P - S
    if ptr < 21 and ptr > 5:
        return 1
    elif ptr < 36  and ptr >= 21:
        return 2
    elif ptr < 40 and ptr >= 36:
        return 3
    return 0

def get_motif_sequence(starting_pos, ending_pos, motif_read):

    # Mod by 40 to get paylaod number
    starting_index = int(starting_pos / 40)
    ptr = starting_pos
    starting_index *= 3

    ptr %= 40

    starting_index += add_remainder_motif(ptr)

    # Mod by 40 to get paylaod number
    ending_index = int(ending_pos / 40)
    ptr = ending_pos
    ending_index *= 3

    ptr %= 40

    ending_index += add_remainder_motif(ptr)


    return motif_read[starting_index: ending_index]

def motif_search(base_seq: str, motif_picks: List[str]):

    motif_length = len(motif_picks[10]) - 1
    motif_prediction = []
    # Sliding window searching for motif picks
    for i in range(0, len(base_seq) - motif_length, motif_length):
        subseq = base_seq[i: i + motif_length]
        for ind, j in enumerate(motif_picks[8:]):
            if subseq in j:
                motif_prediction.append(ind+1)

    return motif_prediction

def load_info_dict(info_dict_filepath: str) -> dict:
    with open(info_dict_filepath, 'r') as f:
        exp_dict = json.load(f)

    return exp_dict

def unpack_info_dict(
        info_dict: dict) -> Tuple[
            List[str], List[str], List[List[int]]]:
    return info_dict['read_ids'], info_dict['motif_choices'] + info_dict['link_choices'], info_dict['motif_labels'], info_dict['reads_base_level']

In [8]:
fast5_filepath = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\dorado-0.5.1-win64\dorado-0.5.1-win64\bin\testing.fast5"
fast5_raw_data, fast5_read_ids = get_data_from_fast5(fast5_filepath)

In [9]:
info_dict_filepath = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\notebooks\info.json"
fasta_read_ids, motif_choices, motif_labels, reads_base_level = unpack_info_dict(load_info_dict(info_dict_filepath))

In [14]:

def create_synthetic_dataset(
        fast5_read_ids: List[str], fasta_read_ids: List[str], fast5_raw_data: List[Any],
        reads_base_level: List[List[str]], motif_labels: List[List[int]], motif_picks: List[str]) -> pd.DataFrame:

    base_sequences_dataset = []
    motif_sequences_dataset = []
    raw_data_arrs = []
    strand_orientations = []
    fasta_read_ids_df = []

    for ind, read_id in tqdm(enumerate(fast5_read_ids), total=len(fast5_read_ids)):
        split_id = read_id.split('!') # To get the starting and ending character positions for the specific read id
        uid = split_id[1][1:]

        reverse_complemented = split_id[4]
        strand_orientations.append(reverse_complemented)
        
        starting_pos, ending_pos = int(split_id[2]), int(split_id[3])
        index = fasta_read_ids.index(uid)
        base_sequence_dataset = reads_base_level[index][starting_pos: ending_pos]
        motif_sequence_dataset = get_motif_sequence(starting_pos, ending_pos, motif_labels[index])

        motif_sequence_no_spacers = [i for i in motif_sequence_dataset if i < 9]

        #motif_search_prediction = motif_search(base_sequence_dataset, motif_picks)
        
        fasta_read_ids_df.append(fasta_read_ids[index])
        base_sequences_dataset.append(base_sequence_dataset)
        motif_sequences_dataset.append(motif_sequence_no_spacers)
        raw_data_arrs.append(fast5_raw_data[ind])

    df = pd.DataFrame()

    df['motif_seq'] = motif_sequences_dataset
    df['base_seq'] = base_sequences_dataset
    df['squiggle'] = raw_data_arrs
    df['fasta_read_id'] = fasta_read_ids_df
    df['fast5_read_id'] = fast5_read_ids
    df['orientation'] = strand_orientations

    return df


In [15]:
df = create_synthetic_dataset(
    fast5_read_ids=fast5_read_ids, fasta_read_ids=fasta_read_ids, fast5_raw_data=fast5_raw_data,
    reads_base_level=reads_base_level, motif_labels=motif_labels, motif_picks=motif_choices
)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [16]:
df

,motif_seq,base_seq,squiggle,fasta_read_id,fast5_read_id,orientation
0,"[3, 4, 7, 3, 4, 7, 8, 4, 5, 6, 8, 2, 5, 6, 7, ...",AACAATCCGGGTTCCGTACGTCTTTGACTGTCTTTGACTCCGTATA...,"[529, 533, 535, 532, 529, 542, 529, 529, 542, ...",59bc0cd0-f0f9-47ee-9804-2a86031d74bf,S1_1!>59bc0cd0-f0f9-47ee-9804-2a86031d74bf!371...,+
1,"[4, 6, 2, 4, 5, 6, 2, 4, 5, 8, 2, 4, 5, 6, 2, ...",TTCCGCTCCGTATAAGTCAATACGCGTGTATTCCGCTGTATTCCGC...,"[510, 513, 509, 515, 516, 512, 510, 502, 496, ...",63584c55-26fd-42b9-a36b-c24df5bd4568,S1_10!>63584c55-26fd-42b9-a36b-c24df5bd4568!56...,+
2,"[6, 7, 1, 3, 4, 6, 5, 6, 7, 8, 2, 3, 6, 8, 3, ...",TGTGATGCCTGCGGTTCGCGTCTTTGACTGTCTTTGACTCGAATGA...,"[485, 481, 490, 488, 505, 490, 487, 490, 501, ...",736c8a8a-1ada-489b-8be4-045e7f1393c8,S1_100!>736c8a8a-1ada-489b-8be4-045e7f1393c8!4...,+
3,"[7, 8, 1, 5, 6, 7, 1, 3, 5, 6, 2, 5, 6, 8, 1, ...",AATGACCTCACGTTCAATGTATTCCGCTGTATTCCGCTCTTCAGAA...,"[552, 537, 536, 550, 537, 546, 534, 534, 545, ...",94e1ff76-141b-42d4-a653-93b57af6e695,S1_1000!>94e1ff76-141b-42d4-a653-93b57af6e695!...,-
4,"[5, 8, 1, 4, 6, 8, 2, 3, 5, 7, 2, 4, 7, 8, 3, ...",GTATACTATAAAATACTATAAATTGATAGGAAGTGAGAATCAATAC...,"[472, 472, 479, 473, 466, 471, 471, 531, 529, ...",8bcfb51e-3638-43ab-a537-ba87297c5af7,S1_10000!>8bcfb51e-3638-43ab-a537-ba87297c5af7...,+
...,...,...,...,...,...,...
19995,"[2, 4, 8, 2, 3, 6, 7, 1, 2, 4, 7, 2, 3, 4, 6, ...",GTGCTATTCCGTAATCTAACGCATTCGTTCCGGCTATTCCGTGCTA...,"[462, 458, 462, 453, 465, 466, 464, 455, 461, ...",9b20d3e5-e129-4c8b-ab29-f7b1af3eb227,S1_9995!>9b20d3e5-e129-4c8b-ab29-f7b1af3eb227!...,+
19996,"[7, 8, 2, 4, 7, 8]",ACGAACTTGAACGAACTCGAATGACCTCACGTTCAATTGAACGAAC...,"[531, 524, 530, 520, 524, 516, 527, 516, 515, ...",627e56e0-5839-44cc-8566-68dd531c2148,S1_9996!>627e56e0-5839-44cc-8566-68dd531c2148!...,-
19997,"[4, 7, 8, 1, 3, 7, 8, 1, 6, 7, 8, 1, 2, 5, 6, ...",TTCCGCTCCGTATAAGTCAATACGCGTGTATTCCGCTGTATTCCGC...,"[512, 504, 519, 514, 518, 520, 509, 499, 516, ...",40c3f16a-3531-4956-b2e0-43896acceca4,S1_9997!>40c3f16a-3531-4956-b2e0-43896acceca4!...,-
19998,"[1, 3, 5, 6, 1, 3, 4, 6, 1, 3, 4, 8, 1, 4, 5, 7]",CGCTACGCTGAGACGATATTGACTCTTTGACTTTACGCTGAGACAC...,"[529, 520, 531, 531, 529, 532, 541, 547, 547, ...",edd826ec-3d7d-4661-9757-aabbbcefeb2c,S1_9998!>edd826ec-3d7d-4661-9757-aabbbcefeb2c!...,+


In [17]:
df.to_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\synthetic\pickled_datasets\no_spacers.pkl")

In [23]:
basecalled_df = pd.read_csv(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\basecalled.csv")

In [24]:
basecalled_df

,Unnamed: 0,read_ids_fasta,read_ids_fast5,base_predictions
0,0,a990ccf4-a3a7-4758-a85b-620a4a4d952,S1_10007!>a990ccf4-a3a7-4758-a85b-620a4a4d952b...,TCCGGGTTCCGTACGCTATTCCGTGCTATTCCGTCCGTATAAGTCA...
1,1,487a304e-7eae-46d7-b3d5-a8e56044e24e,S1_10012!>487a304e-7eae-46d7-b3d5-a8e56044e24e...,TTGCATTGAACGTGAGGTCATTCGCGTTAATTGCCGTTAATTGCGC...
2,2,911ca32d-2c05-4bf9-8878-6e0d8149c6f7,S1_10011!>911ca32d-2c05-4bf9-8878-6e0d8149c6f7...,CTACGCTGAGACAGACAATCGGGTTCCGTACACGCTGAGACACGCT...
3,3,59bc0cd0-f0f9-47ee-9804-2a86031d74bf,S1_1!>59bc0cd0-f0f9-47ee-9804-2a86031d74bf!371...,ATCCGGGTTCCGTACGTCTTTGACTGTCTTTGACTCCGTAAGTCAA...
4,4,3520c848-75d8-4ace-95c3-c9a76a406225,S1_10015!>3520c848-75d8-4ace-95c3-c9a76a406225...,ATTGCTATGATGAACTTTTCTGAAGCGTTAATTGCCGTTAATTGCT...
...,...,...,...,...
19995,19995,9b20d3e5-e129-4c8b-ab29-f7b1af3eb22,S1_9995!>9b20d3e5-e129-4c8b-ab29-f7b1af3eb227!...,ATTCGTAATCTAACGCGTTCGTTCCGGCTATTCCGTCTTTATTCCG...
19996,19996,8dc9a386-ffdc-4329-9337-7dea8fcc9465,S1_999!>8dc9a386-ffdc-4329-9337-7dea8fcc9465!3...,ATTGCTGATTCTCGCTTCCTATCAACGTTAATTGCCGTTAATTGCG...
19997,19997,40c3f16a-3531-4956-b2e0-43896acceca4,S1_9997!>40c3f16a-3531-4956-b2e0-43896acceca4!...,ATTGCATTGAACGTGAGTCATTCGCGTTAATTGCCGTTAGTGCACG...
19998,19998,edd826ec-3d7d-4661-9757-aabbbcefeb2c,S1_9998!>edd826ec-3d7d-4661-9757-aabbbcefeb2c!...,CATGAGACGATATTGACTCTTTGACTTACGCTGAGACACGCTGAGA...


In [33]:
basecalled_df = basecalled_df.rename(columns={"read_ids_fast5": "fast5_read_id"})

In [ ]:
basecalled_df.merge(df)

In [ ]:

merged_df = pd.merge(basecalled_df, df, on='fastb'
'5_read_id')

In [38]:
merged_df = merged_df.drop(columns=['fasta_read_id'])

In [72]:
for row in merged_df.iterrows():
    row = row[1]
    base_prediction = row['base_predictions']
    base_seq = row['base_seq']
    orientation = row['orientation']

    if orientation == '-':
        base_prediction = reverse_complement(base_prediction)

    aligner = Align.PairwiseAligner()
    aligner.mode = "local"
    alignments = aligner.align(base_prediction, base_seq)
    print(alignments[0].counts())
    print(distance(base_prediction, base_seq))
    print(orientation)
    print()


AlignmentCounts(gaps=74, identities=1180, mismatches=0)
69
+

AlignmentCounts(gaps=5, identities=247, mismatches=0)
12
-

AlignmentCounts(gaps=27, identities=626, mismatches=0)
28
+

AlignmentCounts(gaps=46, identities=872, mismatches=0)
41
+

AlignmentCounts(gaps=36, identities=706, mismatches=0)
34
-

AlignmentCounts(gaps=14, identities=248, mismatches=0)
18
-

AlignmentCounts(gaps=24, identities=314, mismatches=0)
21
+

AlignmentCounts(gaps=39, identities=1018, mismatches=0)
39
+

AlignmentCounts(gaps=34, identities=682, mismatches=0)
35
-

AlignmentCounts(gaps=52, identities=574, mismatches=0)
43
+

AlignmentCounts(gaps=25, identities=263, mismatches=0)
25
-

AlignmentCounts(gaps=45, identities=694, mismatches=0)
42
-

AlignmentCounts(gaps=26, identities=717, mismatches=0)
28
-

AlignmentCounts(gaps=59, identities=892, mismatches=0)
54
-

AlignmentCounts(gaps=34, identities=425, mismatches=0)
33
+

AlignmentCounts(gaps=41, identities=692, mismatches=0)
37
-

AlignmentCounts(gaps=57

KeyboardInterrupt: 

In [57]:
df.to_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\synthetic\pickled_datasets\rc.pkl")

In [23]:
import matplotlib.pyplot as plt